<h1> IMPORTING LIBRARIES <h1>

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyodbc
import time

<h1>  Defining the role to be searched <h1>

In [5]:
#DEFINING ROLE TO SEARCH
job = input("Which role do you want to search? ").lower()
job_treated = job.replace(" ","-")


<h1> Web Scrapping Section <h1>

In [23]:
#SETTING SITE URL
url = f'https://www.vagas.com.br/vagas-de-{job_treated}'

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'}
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')


#LAST PAGE
last_page = 1
for page in soup.find_all('a',class_ = 'btMaisVagas btn'):
    last_page = page.get('data-total').strip()
            

#NUMBER OF VACANCIES
numero_total_anuncios = soup.find('div',class_= 'faixa clearfix').get_text()[0:4].strip()
print(f"Existem {numero_total_anuncios} vagas nesta plataforma.")


#####PAGINATION LOOP
for i in range(1,int(last_page)+1):
    url_pag = f'https://www.vagas.com.br/vagas-de-{job_treated}?pagina={i}'
    site = requests.get(url_pag, 'html.parser')
    soup = BeautifulSoup(site.content, 'html.parser')
        
    ######ODD
    anuncios_impar = soup.find_all('li', class_ = 'vaga odd')
        
    #####EVEN
    anuncios_par = soup.find_all('li', class_ = 'vaga even')
        
        
    with open(f'vacancies-{job_treated}.csv','a',newline='', encoding = 'utf-8') as f:
        for anuncio_impar in anuncios_impar:
                
            'job title'
            job_title = anuncio_impar.find('h2',class_='cargo').get_text().strip()

                
            'company title'
            company_title = anuncio_impar.find('span',class_ = 'emprVaga').get_text().strip()

                
            'level vacancy'
            level_vacancy = anuncio_impar.find('span', class_='nivelVaga').get_text().strip()
                
            'location'
            try:
                location = anuncio_impar.find('span', class_= 'vaga-local').get_text().strip()
            except:
                location = ''
                
            'id vacancy'
            id_vacancy = anuncio_impar.find('a',class_='link-detalhes-vaga').get('id').strip()
            link_vacancy = f'https://vagas.com.br/vagas/{id_vacancy}' 
                
            #CREATING LINES IN CSV FILE
            line = job_title + ";" + company_title + ";" + level_vacancy + ";" + location + ";" + link_vacancy + '\n'
            f.write(line)
    
                
        for anuncio_par in anuncios_par:
                
            'job title'
            job_title = anuncio_par.find('h2',class_='cargo').get_text().strip()

                
            'company title'            
            company_title = anuncio_par.find('span',class_ = 'emprVaga').get_text().strip()

            'level vacancy'
            level_vacancy = anuncio_par.find('span', class_='nivelVaga').get_text().strip()
                
            'location'
            try:
                location = anuncio_par.find('span', class_= 'vaga-local').get_text().strip()
            except:
                location = ''
                
            'id vacancy'
            id_vacancy = anuncio_par.find('a',class_='link-detalhes-vaga').get('id').strip()
            link_vacancy = f'https://vagas.com.br/vagas/{id_vacancy}'
                
            #CREATING LINES IN CSV FILE
            line = job_title + ";" + company_title + ";" + level_vacancy + ";" + location + ";" + link_vacancy + '\n'
            f.write(line)

            
    df = pd.read_csv(f'vacancies-{job_treated}.csv',on_bad_lines='skip',  delimiter =";", names =  ['Cargo','Empresa','Nivel','Localidade','Link da Vaga'])   
    df.to_excel(f'vacancies-{job_treated}.xlsx') 

Existem 102 vagas nesta plataforma.


<h1> Importing data to Azure SQL <h1>

In [14]:
t1 = time.time()

server = 'server' #write server
database = 'database' #write base
username = 'username' #write username
password = 'password' #write password
        
#IN THIS CASE I HAVE CONNECTED SETTLING CLOUD PARAMETERS THAT MAYBE YOU WILL CAN SETTLE DIFFERENTLY ACCORDING TO THE CLOUD PROVIDER OR THE SERVICE THAT YOU SCRAPPING REQUIRES. IN THIS SPECIFIC EXAMPLE I HAVE SETTLED EXACTLY LIKE IT CAN BE SEEING BELOW. ALSO I'M SHARING A GITHUB WITH TIPS TO SET THE PARAMETERS.
#https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-SQL-Server-from-Windows
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password+';ENCRYPT=Yes; fast_executemany=True;trusted_connection=no')
cursor = cnxn.cursor()

#LOOP
for i in range(len(df)):
    cargo = df.loc[i,'Cargo']
    empresa = df.loc[i,'Empresa']
    nivel = df.loc[i,'Nivel']
    localidade = df.loc[i,'Localidade']
    link = df.loc[i,'Link da Vaga']

 
    script = '''insert into Vagascom (CargoVaga, Empresa, Senioridade, Localidade, LinkDaVaga) values ('''

    data = "'" + cargo + "','" + empresa + "','" + nivel + "','" + localidade + "','" + link + "')"        

    query = script + data  
    
    try:
        cursor.execute(query)
        cursor.commit()
    except:
        pass

cursor.close()

tempoExec = time.time() - t1
print("Execution Time: {} seconds".format(tempoExec))

Execution Time: 37.48806548118591 seconds
